# Animation Domination

- 150 years of et - now in an exciting video


In [ ]:
import fsspec
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

In [ ]:
# return all the COGS for a given year
def get_tifs(last_prefix):
    files = fs.ls(f'dev-et-data/enduser/DelawareRiverBasin/{last_prefix}')
    return [f for f in files if f.endswith('tif')]

In [ ]:
tif_list = get_tifs('drb150cAnnual')

In [ ]:
tif_list

In [ ]:
import rasterio
import xarray as xr

#returns an xarray DA
def overview_ugly_xr_read(s3_tif_object):
    da = xr.open_rasterio(f's3://{s3_tif_object}')
    src = rasterio.open(f's3://{s3_tif_object}', 'r')
    #nsub = 4  # original scale factor
    nsub = 16
    nsub_half = int(nsub/2)
    data = src.read(out_shape=(1, int(src.height / nsub), int(src.width / nsub)))
    DA = xr.DataArray(data, dims=da.dims, coords=[da.band, da.y[nsub_half:-nsub_half:nsub], da.x[nsub_half:-nsub_half:nsub]])
    DA.attrs = da.attrs
    return(DA)
                        

In [ ]:
%%time
how_many=150
dalist =[overview_ugly_xr_read(tif) for tif in tif_list[:how_many]]

da = xr.concat(dalist, dim='band')
da = da.rename({'band':'year'})

In [ ]:
da = da.assign_coords(year=range(1950,1950+how_many))

In [ ]:
da

In [ ]:
import pickle

with open('drb150c_xarray.pickle', 'wb') as handle:
    pickle.dump(da, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import hvplot.xarray

img_figure = da.hvplot.image(dynamic=False,rasterize=True,grid=True, clim=(0,1000),invert=False, width=400, height=500, cmap='viridis')

In [ ]:
#img_figure.hist()

http://xarray.pydata.org/en/stable/indexing.html

In [ ]:
da.loc[1950:1970,:,:]

In [ ]:
img_fig = da.loc[1999:2020,:,:].hvplot.image(dynamic=False,rasterize=True,grid=True, clim=(0,1000),invert=False, width=400, height=500, cmap='viridis')

# this can take a few minutes ... 

- please enjoy the music while we prepare your hvplot

- da doo run run da doo run run ....

In [ ]:
img_fig.hist()

In [ ]:
# import pickle

# with open('drb150c_xarray.pickle', 'wb') as handle:
#     pickle.dump(da, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
!ls -lh drb*

In [ ]:
da

In [ ]:
dir(da)